In [1]:
import nltk
nltk.download('all',halt_on_error=False)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [2]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
% matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# BeautifulSoup libraray
from bs4 import BeautifulSoup 

import re # regex

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,Dropout
from keras.models import Model
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split


Using TensorFlow backend.


**1. Prepare train and test datasets:**

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
spams = pd.read_table("drive/My Drive/data/SMSSpamCollection.txt",sep="\t",header=0)
messages = pd.read_table("drive/My Drive/data/SMSSpamCollection.txt",sep="\t")


In [37]:
#type of the object
print(type(spams))
#size of the dataset
print(spams.shape)

<class 'pandas.core.frame.DataFrame'>
(5572, 2)


In [38]:
spams.head(10)

,classe,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [40]:
spams.describe

<bound method NDFrame.describe of      classe                                            message
0       ham  Go until jurong point, crazy.. Available only ...
1       ham                      Ok lar... Joking wif u oni...
2      spam  Free entry in 2 a wkly comp to win FA Cup fina...
3       ham  U dun say so early hor... U c already then say...
4       ham  Nah I don't think he goes to usf, he lives aro...
...     ...                                                ...
5567   spam  This is the 2nd time we have tried 2 contact u...
5568    ham               Will ü b going to esplanade fr home?
5569    ham  Pity, * was in mood for that. So...any other s...
5570    ham  The guy did some bitching but I acted like i'd...
5571    ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

In [43]:

#subdivision into train and test sets
from sklearn.model_selection import train_test_split
spamsTrain, spamsTest =train_test_split(spams,train_size=3572,random_state=1,stratify=spams['classe'])
#frequency distribution of the class attribute
#train set
freqTrain = pd.crosstab(index=spamsTrain["classe"],columns="count")
print(freqTrain/freqTrain.sum())
#test set
freqTest = pd.crosstab(index=spamsTest["classe"],columns="count")
print(freqTest/freqTest.sum())

col_0      count
classe          
ham     0.865901
spam    0.134099
col_0   count
classe       
ham     0.866
spam    0.134


**2. Text pre-processing:**

In [44]:
import re
import nltk
nltk.download('stopwords')
#used once to download stopwords.
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0,len(messages['message'])):
#2/3
  smsDocument = re.sub('[^a-zA-Z]',' ',messages['message'][i])
  smsDocument = smsDocument.lower()
  smsDocument = smsDocument.split()
ps = PorterStemmer()
smsDocument = [ps.stem(word) for word in smsDocument if not word in
set(stopwords.words('english'))]
smsDocument = ' '.join(smsDocument)
corpus.append(smsDocument)
smsDocuments = pd.Series(corpus)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**3. Feature extraction:**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

#bag of words
parseur = CountVectorizer()

In [0]:

#create the document term matrix
XTrain = parseur.fit_transform(spamsTrain['message'])

In [48]:

#number of tokens
print(len(parseur.get_feature_names()))


6789


In [49]:
#list of tokens
print(parseur.get_feature_names())


['00', '000', '000pes', '0089', '01223585236', '01223585334', '0125698789', '02', '0207', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07046744435', '07090298926', '07099833605', '07123456789', '0721072', '07732584351', '07734396839', '07753741225', '0776xxxxxxx', '07781482378', '07786200117', '077xxx', '07808247860', '07821230901', '07880867867', '07946746291', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986906', '08006344447', '0808', '08081263000', '08081560665', '0825', '083', '08448714184', '0845', '08452810073', '08452810075over18', '0870', '08700435505150p', '08700469649', '08700621170150p', '08701213186', '08701237397', '08701417012', '087016248', '08701752560', '087018728737', '0870241182716', '08702840625', '08704439680', '08704439680ts', '08706091795', '0870737910216yrs', '08707500020', '08707509020', '0870753331018', '08707808226', '0870

In [0]:
#transform the sparse matrix into a numpy matrix
mdtTrain = XTrain.toarray()


In [51]:
#type of the matrix
print(type(mdtTrain))
#size of the matrix
print(mdtTrain.shape)


<class 'numpy.ndarray'>
(3572, 6789)


In [52]:
#frequency of the terms
freq_mots = np.sum(mdtTrain,axis=0)
print(freq_mots)


[ 7 17  1 ...  1  1  1]


In [53]:
#arg sort
index = np.argsort(freq_mots)
print(index)


[3394 3804 3803 ... 5962 6061 6756]


In [54]:
#print the terms and their frequency
imp = {'terme':np.asarray(parseur.get_feature_names())[index],'freq':freq_mots[index]}
print(pd.DataFrame(imp))

           terme  freq
0          kaiez     1
1        mandara     1
2          manda     1
3     manchester     1
4         manage     1
...          ...   ...
6784          in   584
6785         and   645
6786         the   831
6787          to  1439
6788         you  1495

[6789 rows x 2 columns]


In [55]:
# TF-IDF Model
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(mdtTrain)
print ("IDF:",tfidf.idf_)
tf_idf_matrix = tfidf.transform(mdtTrain)
print (tf_idf_matrix.todense())

IDF: [7.10171932 6.34794751 8.48801368 ... 8.48801368 8.48801368 8.48801368]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**4. Model training and prediction (Classification):**

In [57]:
#*** train the classifier
#import the class Logistic Regression
from sklearn.linear_model import LogisticRegression
3/3
#instatiate the object
modelFirst = LogisticRegression()
#perform the training process
modelFirst.fit(mdtTrain,spamsTrain['classe'])


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
#size of coefficients matrix
print(modelFirst.coef_.shape)


(1, 6789)


In [59]:
#intercept
print(modelFirst.intercept_)


[-4.85725641]


In [0]:
#generate the document term matrix for the test set
#using the object learned from the train set
#create the document term matrix
mdtTest = parseur.transform(spamsTest['message'])

In [61]:
#size of the matrix
print(mdtTest.shape)
#prediction for the test set
predTest = modelFirst.predict(mdtTest)

(2000, 6789)


**5. Model evaluation:**

In [63]:
#import the metrics class for the performance measurement
from sklearn import metrics
#confusion matrix
mcTest = metrics.confusion_matrix(spamsTest['classe'],predTest)
print(mcTest)

[[1728    4]
 [  39  229]]


In [66]:

#recall
print('recall:',metrics.recall_score(spamsTest['classe'],predTest,pos_label='spam'))
#precision
print('precision:',metrics.precision_score(spamsTest['classe'],predTest,pos_label='spam'))
#accuracy rate
print('accuracy rate:',metrics.accuracy_score(spamsTest['classe'],predTest))
#F1-Score
print('F1 score :',metrics.f1_score(spamsTest['classe'],predTest,pos_label='spam'))

recall: 0.8544776119402985
precision: 0.9828326180257511
accuracy rate: 0.9785
F1 score : 0.9141716566866267
